In [53]:
import torch
from transformers import BartForConditionalGeneration, BartConfig, BartTokenizerFast

In [54]:
model_class = BartForConditionalGeneration
config_class = BartConfig
model_fp = "facebook/bart-base"

config = config_class.from_pretrained(model_fp)

model = model_class.from_pretrained(model_fp)
tokenizer = BartTokenizerFast.from_pretrained(model_fp)

In [55]:
example_english_phrase = "Hello world "
batch = tokenizer(example_english_phrase, return_tensors="pt")

In [56]:
tokenizer.batch_decode(batch["input_ids"])

['<s>Hello world </s>']

In [57]:
def get_prefix_allowed_tokens_fn(allowed_seqs):
    allowed_ids = tokenizer(allowed_seqs)["input_ids"]
    allowed_ids = [[2] + allowed_id for allowed_id in allowed_ids]
    print(allowed_ids)

    def prefix_constraining_fn(batch_id, input_ids):
        print(batch_id)
        print(input_ids)
        if len(input_ids.size()) == 1:
            prefix_len = len(input_ids)
            prefix = input_ids.tolis
        elif len(input_ids.size()) == 2:
            prefix_len = len(input_ids[batch_id])
        possible_conts = []
        for allowed_id in allowed_ids:
            if len(allowed_id) > prefix_len:
                if input_ids[batch_id] == allowed_id[:prefix_len]:
                    possible_conts.append(allowed_id[prefix_len])

        print(possible_conts)
        return possible_conts

    return prefix_constraining_fn

In [58]:
prefix_constraining_fn = get_prefix_allowed_tokens_fn(allowed_seqs)

NameError: name 'allowed_seqs' is not defined

In [ ]:
generated_ids = model.generate(
    batch["input_ids"],
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,  # len(tokenizer) - 1,
    bos_token_id=tokenizer.bos_token_id,
    prefix_allowed_tokens_fn=prefix_constraining_fn,
    #                                num_beams=2,
)

In [ ]:
print(tokenizer.batch_decode(generated_ids))